In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import time
import statsmodels.api as sm
from sklearn import linear_model
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import mean_squared_error
from statsmodels.regression.quantile_regression import QuantReg

from sklearn.preprocessing import StandardScaler

import os

from sklearn.model_selection import train_test_split

In [3]:
import random as rd

In [2]:
df = pd.read_csv('clean_plan.csv')

In [3]:
df.columns

Index(['Unnamed: 0', 'date', 'content'], dtype='object')

### Google Search

In [ ]:
# cx : f31c7fa83bc80772c


In [6]:
def get_keys(path='keys.txt'):
    with open(path,'r') as f:
        lines = f.readlines()
        keys={}
        for line in lines:
            items = line.split(':',maxsplit=1)
    #         print("items :",items)
            if '\n' in items[1]:
                items[1] = items[1].replace('\n','')
            keys[items[0]]=items[1]
#             cmd = "%s = \'%s\'"% (items[0],items[1])
#             exec(cmd)
            print("%s was loaded"%items[0])
    return keys

In [7]:
keys = get_keys()

cs was loaded
api_key was loaded


In [39]:
# !pip install google-api-python-client

In [11]:
!pip install google-api-python-client

  Using cached google_api_python_client-1.12.8-py2.py3-none-any.whl (61 kB)
  Using cached google_auth_httplib2-0.0.4-py2.py3-none-any.whl (9.1 kB)
  Using cached httplib2-0.18.1-py3-none-any.whl (95 kB)
  Using cached uritemplate-3.0.1-py2.py3-none-any.whl (15 kB)
  Using cached googleapis_common_protos-1.52.0-py2.py3-none-any.whl (100 kB)


In [12]:
from googleapiclient.discovery import build
import json

In [13]:
my_api_key = keys['api_key']
my_cse_id = keys['cs']  
def google_search(search_term, api_key, cse_id, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
    return res

In [44]:
# //구글로 검색하기
search_result = google_search("Python Google Custom Search", my_api_key, my_cse_id)

In [50]:
# search_result :  save results as json 
with open('search_result_all.json', 'w', encoding='utf-8') as f:
    json.dump(search_result, f, ensure_ascii=False, indent=4)
# items=search_result['items']

#### fill Table Nan into random values 

In [7]:
faulty = pd.DataFrame(df.content.value_counts())[:5]

In [8]:
f_con = faulty.index.tolist()

In [9]:
f_con

['멘토링 멘토링',
 '기업연계 프로젝트 프로젝트 개발',
 '파이널 프로젝트 프로젝트 개발',
 '기업연계 프로젝트 프로젝트 시스템 설계',
 '특강 특강']

In [37]:
df['content'] = df['content'].apply( lambda x : np.nan if (x in f_con) else x )

In [66]:
rd.sample(contents,k=1)[0]

'자연어 처리 자연어 처리'

In [44]:
contents = df.content.unique().tolist()

In [57]:
contents.remove(np.nan)

ValueError: list.remove(x): x not in list

In [68]:
# repl_nan = [rd.sample(contents,k=1)[0] for _ in range(df.content.isna().sum())   ]

In [150]:
df['content'] = df.content.apply(lambda x : rd.sample(contents,k=1)[0] if x is np.nan else x )

#### json format 'item'>[num]>  
#### <pre>              title,link,snippet</pre>

#### Extract title , link 

In [156]:
del df['Unnamed: 0']

In [160]:
df['links']  = [ [] for _ in range(len(df) ) ]
df['titles'] = [ [] for _ in range(len(df) ) ]

In [184]:
pd.concat([pdf(df.content.shift(1)),pdf(df.content)],axis=1)

,content,content
0,NaN,기계학습 Ensemble기법
1,기계학습 Ensemble기법,기계학습 강화학습
2,기계학습 강화학습,기계학습 강화학습2
3,기계학습 강화학습2,딥러닝 기본 딥러닝 이해
4,딥러닝 기본 딥러닝 이해,딥러닝 기본 DNN
5,딥러닝 기본 DNN,딥러닝 기본 DNN
6,딥러닝 기본 DNN,딥러닝 기본 CNN
7,딥러닝 기본 CNN,딥러닝 기본 CNN
8,딥러닝 기본 CNN,딥러닝 기본 RNN
9,딥러닝 기본 RNN,딥러닝 기본 RNN


In [186]:
df_dpc = df.content.shift(1) == df.content

In [207]:
df.loc[df_dpc.replace({True:False,False:True}),'count']=0

In [248]:
df.rename(columns = {'count':'num'},inplace=True)

In [257]:
tt = df.num.copy()

### df.num 생성 

In [272]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   date     91 non-null     int64  
 1   content  91 non-null     object 
 2   links    91 non-null     object 
 3   titles   91 non-null     object 
 4   num      76 non-null     float64
dtypes: float64(1), int64(1), object(3)
memory usage: 3.7+ KB


In [321]:
np.isnan(df.num[5])

True

In [299]:
type(df.num.iloc[5])

numpy.float64

In [326]:
def func(con):
    ctnt = con.copy()
    for x in range(len(con)):
        if np.isnan(ctnt.iloc[x] ) :
            con.iloc[x] = con.iloc[x-1]+1
    return con

In [327]:
df['num'] = df[['num']].apply(func)

In [330]:
df.to_csv('developed_plan.csv')

### GET MODIFIED PLAN

In [4]:
df = pd.read_csv('developed_plan.csv')

In [14]:
with open('search_result_all.json','r') as f:
    dt = json.load(f)

### scratch title,link,snippet  from google

In [58]:
df['snippets'] = [list() for _ in range(len(df))]

In [132]:
for x in range(len(df)) :
    day = df.iloc[x]
    content = day['content']
    count   = day['num']*10
    result = google_search(content,keys['api_key'],keys['cs'],start=count)
    with open('scratched/%s_%d_%d.json'%(content,x,day['num']),'w') as f :
        json.dump(result,f)
    
    links    = []
    titles   = []
    snippets = []
    
#     search = result['items']
    for res in result['items'] :
        links   .append(res['link'])
        titles  .append(res['title'])
        snippets.append(res['snippet'])
    print(x,'titles :',titles[0])        
    df.loc[x,'links']  = ','.join(links)
    df.loc[x,'titles'] = ','.join(titles)
    df.loc[x,'snippet']= ','.join(snippets)

0 titles : 앙상블
1 titles : Improving Quantum Computation with Classical Machine Learning
2 titles : Machine Learning and Visual Computing Laboratory
3 titles : 처음 배우는 딥러닝 수학: 그림으로 이해하고 엑셀로 확인하는 딥러닝 ...
4 titles : [SAS로 딥러닝 시작하기#1]기본 DNN, 배치 정규화를 사용한 DNN ...
5 titles : WO2019074195A1 - A deep learning-based image comparison ...
6 titles : [SAS로 딥러닝 시작하기#1]기본 DNN, 배치 정규화를 사용한 DNN ...
7 titles : Machine Learning Glossary | Google Developers
8 titles : RNN and LSTM - lhparkdeep
9 titles : 모두의 딥러닝 개정 2판: 자연어 처리, GAN, 오토인코더, 전이 학습 등 ...
10 titles : [SAS로 딥러닝 시작하기#1]기본 DNN, 배치 정규화를 사용한 DNN ...
11 titles : 가장 빨리 만나는 딥러닝 with Caffe: 인공지능, 기계 학습, 이제는 딥 ...
12 titles : Cloud Natural Language | Google Cloud
13 titles : KR20070037969A - 자연어처리를 이용한 데이터베이스 검색 시스템 ...
14 titles : KR20170030297A - 자연어 처리 시스템, 자연어 처리 장치, 자연어 ...
15 titles : Cloud Natural Language | Google Cloud
16 titles : 구성원 - hallymnlp
17 titles : 일래스틱서치를 활용한 자연어 처리와 정보검색 세트(에이콘 오픈소스 ...
18 titles : AI Platform 및 BERT를 사용하여 자연어 처리 통합 

#### modify csv

In [5]:
df = pd.read_csv('scratched_plan.csv')

In [7]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'date', 'content', 'links', 'titles',
       'num', 'snippets', 'snippet'],
      dtype='object')

In [8]:
del df['Unnamed: 0']

In [10]:
del df['Unnamed: 0.1']

In [12]:
del df['snippets']

In [43]:
df.snippet = df.snippet.str.replace('\n',' ')

In [51]:
df.snippet.str.split(',')

['머신러닝 앙상블이란 여러개의 머신러닝 모델을 이용해 최적의 답을 찾아내는 기법 이다. 여러 모델을 이용하여 데이터를 학습하고',
 ' 모든 모델의 예측결과를 평균하여\xa0...',
 '본 발명은 앙상블 기계학습 방법을 이용한 답안 채점 방법에 관한 것이다. 이러한  방법은',
 ' a) 다수의 입력 답안에 대하여 자연 언어 처리를 수행하여 다수의 채점 대상 \xa0...',
 'Compilation of key machine-learning and TensorFlow terms',
 ' with beginner- friendly definitions. ... You can create an ensemble via one or more of the  following:.',
 'The machine learning model 32 may be a macro-procedure 36 that combines the  outcomes of the ensemble of micro-procedures 38. Each micro-procedure 38\xa0...',
 '기계 학습 엔진이 제시된다. 개시된 추천 엔진은 공지된 게놈 데이터 세트 및 상응 하는 공지된 임상 결과 데이터 세트에 대해 트레이닝되는 트레이닝된 기계 학습  모델\xa0...',
 '캐글 경진대회지만 리더보드 + 각 앙상블의 타당성/정리 등이 채점 기준입니다 EDA  & Feature ... 이번주 까지 배운 모든 ml모델과 기법 등을 적극 활용해주세요.',
 "일반적으로 데이터 마이닝 및 기계 학습에서 앙상블 기법으로는 'Breman'이 제시한  배깅(Bagging) 기법을 효시로 하여 최근까지 많은 연구가 진행되고 있다.",
 'Bagging Ensemble of Multilayer Perceptrons for Missing Electricity Consumption  ... 전력수요 예측에서 결측치 대체를 위한 다양한 기계 학습 기법의 성능 비교.',
 '앙상블 기법을 활용한 온라인 음식 상

In [143]:
df.drop('Unnamed: 0',axis=1,inplace=True)    

In [147]:
del df['num']

In [52]:
df

,date,content,links,titles,num,snippet
0,20210114,기계학습 Ensemble기법,https://colab.research.google.com/github/teddy...,"앙상블,KR101713487B1 - 앙상블 기계학습 방법을 이용한 서답형 답안 채점...",0.0,머신러닝 앙상블이란 여러개의 머신러닝 모델을 이용해 최적의 답을 찾아내는 기법 이다...
1,20210115,기계학습 강화학습,https://feedproxy.google.com/~r/blogspot/gJZg/...,Improving Quantum Computation with Classical M...,0.0,"Oct 3, 2019 ... Posted by Murphy Yuezhen Niu a..."
2,20210118,기계학습 강화학습2,https://sites.google.com/a/khu.ac.kr/mlvc/clas...,Machine Learning and Visual Computing Laborato...,0.0,기계학습의 수학적 이론을 이해한다. 다양한 딥 ... 최종 프로젝트 발표 선정시 ...
3,20210119,딥러닝 기본 딥러닝 이해,https://books.google.com/books/about/%EC%B2%98...,"처음 배우는 딥러닝 수학: 그림으로 이해하고 엑셀로 확인하는 딥러닝 ...,[SAS...",0.0,2018년 2월 9일 ... 고등학교 수학으로 살펴보는 딥러닝 개념딥러닝 모델의 바...
4,20210120,딥러닝 기본 DNN,http://feedproxy.google.com/~r/sasblogs/~3/UDJ...,"[SAS로 딥러닝 시작하기#1]기본 DNN, 배치 정규화를 사용한 DNN ...,코...",0.0,2020년 10월 23일 ... 예제 1: 기본 심층신경망(DNN) 모델 아키텍처....
...,...,...,...,...,...,...
86,20210522,지능형 IoT 사물인터넷 통신,https://www.google.com/patents/WO2016186456A1?...,WO2016186456A1 - 셀룰러 사물 인터넷 네트워크에서의 통신을 위한 ......,0.0,본 개시는 5G 통신 기술 및 IoT 관련 기술을 기반으로 지능형 서비스 (예를 들...
87,20210525,세미 프로젝트2 스케쥴링과 인프라 구축,https://patents.google.com/patent/KR2019001086...,KR20190010863A - Multiplexing of sub-frames ha...,0.0,The 3G Partnership Project (3GPP) began its jo...
88,20210526,딥러닝 기본 DNN,http://feedproxy.google.com/~r/sasblogs/~3/UDJ...,"[SAS로 딥러닝 시작하기#1]기본 DNN, 배치 정규화를 사용한 DNN ...,코...",0.0,2020년 10월 23일 ... 예제 1: 기본 심층신경망(DNN) 모델 아키텍처....
89,20210527,기업연계 프로젝트 프로젝트 테스트,https://news.google.com/__i/rss/rd/articles/CB...,"과기부, 내년 소재·부품·장비 R&D 예산 두 배 늘린다 - IT조선 > 기업 .....",0.0,2019년 9월 9일 ... 또한 대규모 투자가 필요한 공용 반도체 테스트베드를 4...
